In [ ]:
import aextl
import numpy as np
import pandas
import os
import yaml
import math
from collectResults import *
#import collectResults
from operator import itemgetter
from IPython.display import display


In [ ]:
#probably, the next is not needed
#%run collectResults.py

In [ ]:
configFile = "../experiments/"

allRuns = list()
#allRuns: an object containing all runs
#cfg: object returned from the aext tool
allRuns, cfg = collectAllRuns( configFile )

#TODO: add all required information in allRuns so cfg is not needed
#TODO2: then why not use cfg in the first place?

In [ ]:
# test: print run times for a specific experiment for a specific instance

# 22/05: there are data for rk and dimacs9-BAY but this graph is not in the evaluation set so the data are not gathered
#print( [run['run_time'] for run in allRuns if (run['experiment']=='rk' and run['instance']=='dimacs9-COL') ] )
#print( [run['run_time'] for run in allRuns if (run['experiment']=='kadabra' and run['instance']=='dimacs9-COL') ] )

print( 'possible experiment are:' , set([run['experiment'] for run in allRuns]) )

print( "possible parameter values:\n", list(dict.fromkeys([run['parameters']['iters_per_step'] for run in allRuns if run['instance']=='advogato' and 'iters_per_step' in run['parameters'] ] ) ))

#print( [run['variation'] for run in allRuns if run['instance']=='advogato' and 'iters_per_step' in run['parameters'] ] )

print( allRuns[134] ) 

In [ ]:
# lexicographic sorting

#sort first by experiment/tool name and then by instance name
#probably not needed but better to be sure than sorry
allRuns.sort( key=itemgetter( 'experiment', 'instance') )

print("All runs are", len(allRuns))

## Add the number of nodes and edges of the instance for every run

In [ ]:
#TODO: if a subset of graphs is stored in the file, it will read only from the file 
# and ignore the new graphs
_ = addAttributes( allRuns )

In [ ]:
# store instances attributes to file 
#storeInstToFile( allRuns, 'instancesData.yml' )

## Check if we have data for all input instances
If not, remove the runs that do not contain data for both algorithms

In [ ]:
toBeRemoved = list()

for run in allRuns:
    inst = run['instance']
    algo = run['experiment']
    flag = True
    for run2 in allRuns:
        #there is a run for the same instance but different algorithm
        if run2['instance']==inst and run2['experiment']!=algo:
            flag = False
            break
    if flag:
        #print( run )
        #print( run2 )
        toBeRemoved.append( inst )

toBeRemoved = list(dict.fromkeys(toBeRemoved))        
print( "instances to be removed:", toBeRemoved) 
print("WARNING: this is the evaluation set for which we do not collect rk runs; do not remove")

#do not remove anything
toBeRemoved.clear()

In [ ]:
tmpFilterRuns = list()
runsRemoved = 0
for run in allRuns:
    if len(run)!=0 and run['instance'] not in toBeRemoved:
        tmpFilterRuns.append( run )
        #run.clear()
    else:
        runsRemoved+=1
        #print(run['instance'])
            
print( "runs removed are" , runsRemoved )

In [ ]:
allRuns.clear()
allRuns = tmpFilterRuns.copy()
print( len(allRuns))

## Seperate runs that belong to the tunning set and to the evaluation test
Tunning set contains only runs for kadabra (this is the algorithm we tune).
The two sets have some overlap, that is, the runs for the tunning

In [ ]:
#all parameter values
allParams = list()

for run in allRuns:
    if 'iters_per_step' in run['parameters']:
        allParams.append( run['parameters']['iters_per_step'] )
        allParams = list( dict.fromkeys(allParams) )
        
# turn None to -1
for i in range(len(allParams)):
    if allParams[i]==None:
        allParams[i]=-1
print("all available parameter values are:", allParams)
print("RK does not have a parameter; we set a dummy value to -1")

In [ ]:
# import the tunning parameter value
# this is determined after the tunning phase

import sys
sys.path.insert(0, '../experiments/')
#%run ../experiments/explore.py

In [ ]:
from explore import *
#calculates and prints the tunning parameter
tunedParam, minTuningTime = getTunParam()
minTuningTime = round( minTuningTime, 1 )
print("tuned parameter value is " + str(tunedParam) + ", min time is " + str(minTuningTime) )

In [ ]:
#filter all runs by parameter value
filteredRunsPerParam = dict()

runsRemoved=0

for param in allParams:
    #rk does not have a 'iter_per_step' parameter
    if param==-1:
        filteredRunsPerParam[param]= [ run for run in allRuns if run['experiment']=='rk' ]
    else:
        filteredRunsPerParam[param]= \
        [ run for run in allRuns \
           if 'iters_per_step' in run['parameters'] \
           and run['parameters']['iters_per_step']==param ] #for this param value

#keep only runs from the evaluation test set with the correct tunned parameter
#evaluation set runs are the ones with the tunned parameter + all for rk

#only instances in the evaluation set
#for the runs of the tunnedParameter, filter out the runs of the tuning set
evalRuns = [ 
    run for run in filteredRunsPerParam[tunedParam]  if 'tuning' not in run['set'] ] \
    + [run for run in filteredRunsPerParam[-1] if 'tuning' not in run['set'] 
]

print("Number of runs in the evaluation set are", len(evalRuns) )

In [ ]:
#tunning set runs are only runs of kadabra

#tunning runs are separated by the parameter value
tuningRuns = dict()
tuningSize = 0
for x in allParams:
    if x!=-1: #-1 signifies that runs for RK
        if x==tunedParam:
            #from the runs for the tunned parameter, get only the ones of the tunning set       
            tuningRuns[x] = [ run for run in filteredRunsPerParam[x] if 'tuning' in run['set'] ]
        else:
            tuningRuns[x] = filteredRunsPerParam[x]                 

        print( x, len(tuningRuns[x]), set([ x['instance'] for x in tuningRuns[x]]) )
        tuningSize += len(tuningRuns[x])

print( "Number of runs in the tunning set are", tuningSize )

tuningInstSize = len( set([ x['instance'] for x in tuningRuns[1]]) )
print("size of tuning set is " + str(tuningInstSize) )
#for next assertion: 3 is the repeated runs for the tunning set, 5 is the repeated runs for the 
#evaluation set, |allParams|-2 because we exclude the tunned valued (and -1 for rk)

# in general: tunnigSetSize = |tunningSet|*( repeatForTunSet*(|param|-1) + repeatsForEvalSet )

assert( tuningSize == ( tuningInstSize*(3*(len(allParams)-2)+3)) )  

In [ ]:
print( "Number of kadabra runs in the evaluation set:", len([run['experiment'] for run in evalRuns if run['experiment']=='kadabra']) )
print( "Number of rk runs in the evaluation set:", len([run['experiment'] for run in evalRuns if run['experiment']=='rk']) )
assert( -1 not in tuningRuns ) # only runs of kadabra
#same check as above, TODO: remove?
for tRun in tuningRuns:
    for t in tuningRuns[tRun]:
        assert( t['experiment']=='kadabra')

## Sort lexicographicaly 
by tool name and then by number of nodes. 
This sorting also by size is important to ensure that ratios will be sorted according to instance size

In [ ]:
allRuns.sort( key=itemgetter( 'experiment', 'n') )
evalRuns.sort( key=itemgetter( 'experiment', 'n') )
for tRun in tuningRuns:
    tuningRuns[tRun].sort( key=itemgetter( 'experiment', 'n') )

print(evalRuns[0])
print(evalRuns[51])
print(evalRuns[133])
print(tuningRuns[100][0])
#print(tunningRuns[1][0]['parameters'])
#print(tunningRuns[100][0]['parameters'])

# Extract running times ratios
- We sort the allRuns list lexicographicaly, first by experiment and then by instance size.
- So, when we extract the instances names, these are also sorted by size.
- Similarly, the ratios below are also sorted by the instance size.
- Because of the sorting by names, the ratios are 'tool1_run_time' / 'tool2_run_time', where 'tool1_name'<'tool2_name'. In this case, the ration is kadabra/rk (since 'k'<'r')

## Extract the instances names and sizes.
Instances might not be needed. Sizes will be used for plotting.
Size can be the number of nodes, edges or both

In [ ]:
# all the instances names
#since allRuns are sorted, instances should be sorted by size (=number of nodes)
allInstances = list( dict.fromkeys([ r['instance'] for r in allRuns]) ) #remove duplicates
numInstances = len(allInstances)
abbrvInst = abbreviate( allInstances );

evaluationInst =  list( dict.fromkeys([run['instance'] for run in evalRuns]) ) # if 'tuning' not in run['set']]))
numEvalInst = len(evaluationInst)
tuningInst = list( dict.fromkeys([run['instance'] for run in tuningRuns[1]]) ) # if 'tuning' in run['set']]))
numTunInst = len(tuningInst)
print( ">>> tuning set length",numTunInst  , "and it is:\n", tuningInst )
print( ">>> evaluation set length", numEvalInst, "and it is:\n",evaluationInst )

#the two set do NOT have an empty intersection: tunningSet<evaluationSet
assert( set(evaluationInst).isdisjoint(set(tuningInst)) )

In [ ]:
# extract the sizes of all instances and remove duplicates due to several runs per instance
sizesN = list( dict.fromkeys([ r['n'] for r in evalRuns]) )
sizesM = list( dict.fromkeys([ r['m'] for r in evalRuns]) )
print(sizesN)
sizesNM = [ n*m for n,m in zip(sizesN,sizesM)]
sizesNplusM = [n+m for n,m in zip(sizesN,sizesM)]

# make sure sizes are sorted and of equal size
assert( sorted(sizesN)==sizesN)
assert( len(sizesN)==len(sizesM) )
assert( len(sizesN)==numEvalInst )

print(">>> all evaluation instances set length", len(evaluationInst) , "and they are:\n", [ (inst,n) for inst,n in zip(evaluationInst,sizesN)])

In [ ]:
algos = set([run.experiment.name for run in cfg.discover_all_runs()])
assert( len(algos)==2 )
print("Possible experiment names are", algos)

## Get the mean running times using the arithmentic mean 
to summarize all running times for (tool,instance) in one.


In [ ]:
#in general
#timeToolX = [ r['run_time'] for r in allRuns if r['experiment']=='toolX' ]
# or using a dictionary:
#time['toolX'] = [ r['run_time'] for r in allRuns if r['experiment']=='toolX' ]

In [ ]:
# a dictionary of lists to store all running times for every instance 
# indexed by the instance name: allTimesRK['instanceX'] is a list of all
# the run times with RK for instanceX, len(allTimesRK['instanceX'])=number of repeats

stopTime = 60*60*7 #7 hours

#will be used later for the box and violin plots
allTimesRK = dict()
allTimesKad = dict()

#for one tool and one instance there are several running times
#extract a list per tool and instance, calculate the mean and store it in a list

# the mean running times of all runs for every instance, used for speedup and scatter plot
timeRKMean = list()
timeKadMean = list()
evalBothInst = list() #evaluation instances for both algorithms

#TODO: arithmentic or geometric mean?
for inst in evaluationInst:
    timeRKInst = [ r['run_time'] for r in evalRuns if r['experiment']=='rk' and r['instance']==inst ]
    allTimesRK[inst] = timeRKInst
    #maybe we do not have some instances for RK
    if len(timeRKInst)!=0:
        timeRKMean.append( sum(timeRKInst)/len(timeRKInst))
        evalBothInst.append(inst)
    else:
        #timeRKMean.append( stopTime ) #TODO: what to add here?
        timeRKMean.append( -1 )
    
    timeKadInst = [ r['run_time'] for r in evalRuns if r['experiment']=='kadabra' and r['instance']==inst ]
    allTimesKad[inst] = timeKadInst
    timeKadMean.append( sum(timeKadInst)/len(timeKadInst))

nBoth = len(evalBothInst)
missingInst = numEvalInst - nBoth
onlyKadInst = list( set(evaluationInst)-set(evalBothInst))

assert( len(onlyKadInst)==missingInst )
assert( len(timeRKMean)==numEvalInst )
assert( len(timeRKMean)==len(timeKadMean) )

if nBoth != numEvalInst:
    print("### Warning, we do not have results for all algorithm and all instances")
    print("### we have", nBoth, "for both but all evaluation instances are", numEvalInst)
    print("### instances only for kadabra:", onlyKadInst)
    #evaluationInst = evalBothInst.copy()

# WARNING
we do not have data for the last 2 instances for RK. The following script will work if the instances missing
are the largest ones, i.e. the last ones (if sorted by size). If we have other instances missing, say 
the 5th instance out 10, probably the rest of the code will not work correctly and need adaptation.

## Get the times for the tunning set

In [ ]:
#the mean running time per parameter value
tuningTimeKadMean = dict()
    
for tRun in tuningRuns:
    #18 = 3 repeats for 6 tunning instances
    #125 only for the tunned valued # update: this should not happen now
    print( "param value:", tRun, ", number of runs:", len(tuningRuns[tRun]) )
    
    tuningTimeKadMean[tRun] = list()
    thisRunInst = [x['instance'] for x in tuningRuns[tRun] ] #just for checking
    for inst in tuningInst:
        assert( inst in thisRunInst )
        timeKad = [x['run_time'] for x in tuningRuns[tRun] if x['instance']==inst ]
        tuningTimeKadMean[tRun].append( sum(timeKad)/len(timeKad) )
        
    #this is equal to the evaluation set size
    assert( len(tuningTimeKadMean[tRun])== len(tuningInst) )

In [ ]:
# test print
print("instance " + allInstances[12])
print(allTimesKad[allInstances[12]])

# Various Visualization Plots - Running Time


## After creating the running time ratios, plot them

In section 6.Visualizing Results, about paragraph 4, it is mentioned: 

"The x-axis, in turn, is used for the most relevant parameter describing the instances (e. g., instance size
for a scalability evaluation or a graph parameter for evaluating its influence on the solution quality)."

According to that, the x-axis should have instance sizes and not instance names.


In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

In [ ]:
#create ratios of running time
# rk/kadabra
#ratios = [ tRk/tKad for tRk, tKad in zip(timeRKMean,timeKadMean)]

ratios= list()

for tRk, tKad in zip(timeRKMean,timeKadMean):
    #we do not have data for some instance for RK
    if tRk!=-1:
        ratios.append( tRk/tKad  )
    #print( tRk, tKad)
        
assert( len(ratios)==nBoth )
        
if nBoth != numEvalInst:
    print("###Warning, we do not have results for all algorithm and all instances")
    print("we have", nBoth, "for both but all evaluation instances are", numEvalInst)

In [ ]:
print( np.argmin(ratios), np.argmax(ratios))

geomMean = round( np.array(ratios).prod()**(1.0/len(ratios)), 1 )
minSpeedUp = round( min(ratios), 1)
maxSpeedUp = round(max(ratios), 1)
minSpeedUpInst = evaluationInst[np.argmin(ratios)]
maxSpeedUpInst = evaluationInst[np.argmax(ratios)]
# add the abbreviated instance name
minSpeedUpInst += " ("+abbreviate(evaluationInst)[np.argmin(ratios)]+")"
maxSpeedUpInst += " ("+abbreviate(evaluationInst)[np.argmax(ratios)]+")"

print( "min=", minSpeedUp, minSpeedUpInst )
print( "max=", maxSpeedUp, maxSpeedUpInst )
print( "geometric mean = ", geomMean)

#arithmMean = sum(ratios)/len(ratios)
#print(arithmMean)

fig, ax = plt.subplots(figsize=(15,7))
plt.rcParams.update({'font.size': 25})

#plt.title('Overall speedup of KADABRA over RK for every instance in the evaluation set')
plt.ylabel("algorithmic speedup (log scale)", fontsize=23 )
#plt.xlabel("Evaluation set")
plt.xticks( [i+1 for i in range(nBoth)] , abbreviate(evaluationInst), rotation=40, fontsize=25 )

plt.bar( [i+1 for i in range(nBoth)] ,ratios, width=0.4)
plt.plot( [0, numEvalInst+1], [geomMean, geomMean], label='geometric mean=' + str(geomMean), linewidth=2.5, color='m' )
plt.xlim(left=0, right=nBoth+0.5)

ax.annotate(geomMean, xy=(0, geomMean), xytext=(0, geomMean*1.1), color='m' )

ax.set_yscale('log', basey=2)
plt.ylim(top=700)

plt.legend(loc=2)

#pdf has problems with minus sign in the axes
plt.savefig('../images/speedupPlot.pdf')
plt.savefig('../images/speedupPlot.png')
plt.show()

In [ ]:
if nBoth != numEvalInst:
    print("###Warning, we do not have results for all algorithm and all instances")
    print("we have", nBoth, "for both but all evaluation instances are", numEvalInst)
    
# there are many instances with the same diameter
diams = list() 
for i in range(nBoth):
    inst = evaluationInst[i]
    for run in evalRuns:
        if run['instance']==inst:
            diams.append( run['diam'] )
            break

#actually. there is no need to sort...
diamRatios = [ x for _,x in sorted( zip(diams,ratios), key=lambda pair: pair[0]) ]

print(len(diamRatios))
print(len(diams))

fig, ax = plt.subplots(figsize=(15,8))
plt.rcParams.update({'font.size': 22})

plt.title('Overall speedup of KADABRA over RK for every instance in the evaluation set')
plt.ylabel("running time speedup")
plt.xlabel("instance diameter")

#plt.scatter( diams , timeKadMean, marker='o', s=50 )
#plt.scatter( diams , timeRKMean, marker='o', s=50 )

#plt.scatter( sorted(diams) , diamRatios, marker='o', s=50 )
#plt.scatter( diams , ratios, marker='o', s=30 )


kadPlt = plt.scatter( diams, timeKadMean[:-missingInst], s=150, label='kadabra' )
#kadPlt2 = plt.scatter( sizesNplusM[-1:], timeKadMean[-1:], s=150, marker='s', label='kadabra only', color=u'#1f77b4')
RKplt = plt.scatter( diams, timeRKMean[:-missingInst], s=150, marker='P', label='rk' )

ax.set_yscale('log', basey=2)
ax.set_xscale('log', basex=2)

#plt.savefig('../images/speedupPlot.pdf')
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
ax.set_title("all speedups")
plt.ylabel("running time speedup")
plt.xlabel("TODO: x-axis would be how many instances have this speed up?")

ax.violinplot(ratios)

#maybe we could combibe it with something like this:
plt.bar( [i/150+1 for i in range(nBoth)], sorted(ratios,reverse=True) ,width=0.004 )
#plt.scatter( [i/150+1 for i in range(numEvalInst)], sorted(ratios,reverse=True) ,s=15 )

plt.show()

In [ ]:
f, axarr = plt.subplots(4, 3, figsize=(15, 15))
fig.subplots_adjust(hspace = 1.5, wspace=1.5)

axarr = axarr.ravel()
plt.rcParams.update({'font.size': 10})
plt.setp( axarr, xticks=[i+1 for i in range(numEvalInst)], xticklabels=abbreviate(tuningInst) )
#get speedups for the tunning set, for different parameter values
tunRatios = dict()
tunRunTime = dict()
pos = 0

for v in tuningRuns:
    # rk/kadabra
    tunRatios[v] = [ tRk/tKad for tRk, tKad in zip(timeRKMean,tuningTimeKadMean[v])]
    axarr[pos].set_title("speedups for param value=" +str(v) )
    tunRunTime[v] = [ tKad for tKad in tuningTimeKadMean[v]]
    print(v, np.mean(tunRunTime[v]) ) #, tunRunTime[v] )
    #axarr[pos].boxplot( tunRatios[v] )
    #axarr[pos].violinplot( tunRatios[v] )
    axarr[pos].bar( [i+1 for i in range(len(tuningInst))] ,tunRunTime[v], width=0.3)
    pos += 1

plt.show()

In [ ]:
from matplotlib.pyplot import cm

fig, ax = plt.subplots(figsize=(16,8))
plt.rcParams.update({'font.size': 18})

#plt.xlim(right=6)
print(tuningInst)

tuningSetSizes = list( dict.fromkeys([x['n'] for x in tuningRuns[1]]) )
print(tuningSetSizes)

colors=cm.rainbow(np.linspace(0,1,len(allParams)-1))
pos = 0

for i in sorted(allParams):
    if i==-1:
        continue
#    plt.plot( tuningSetSizes , tunRunTime[i], marker='v', markersize=10, label=i , color=colors[pos], linestyle='' )
    pos+=1

plt.plot( tuningSetSizes , tunRunTime[10], marker='s', markersize=12, label='c=10' , color=colors[1], linestyle='' )
plt.plot( tuningSetSizes , tunRunTime[1000], marker='x', markersize=12, label='c=1000' , color=colors[2], linestyle='' )
plt.plot( tuningSetSizes , tunRunTime[4375], marker='v', markersize=12, label='c=4375' , color=colors[9], linestyle='' )
plt.plot( tuningSetSizes , tunRunTime[10000], marker='o', markersize=12, label='c=10000' , color=colors[6], linestyle='' )
    
ax.set_yscale('log', basey=2)
ax.set_xscale('log', basex=2)

#plt.xticks( [math.pow(2,math.log(s,2)) for s in tunningSetSizes], abbreviate(tunningInst_X) )
plt.xticks( tuningSetSizes, abbreviate(tuningInst), rotation=40, fontsize=22  )
    
plt.legend(loc='best',  bbox_to_anchor=(1, 1))
plt.savefig('../images/allTunParams.pdf')
plt.savefig('../images/allTunParams.png')
plt.show()

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

#get geometric mean of all tunning parameters
fig, ax = plt.subplots(figsize=(11,4))
plt.rcParams.update({'font.size': 18})
plt.rcParams['axes.linewidth'] = 1 

tunParamGeomMean = list()
tunParamArithmMean = list()

for i in sorted(allParams):
    if i==-1:
        continue
    tunParamGeomMean.append( np.array(tunRunTime[i]).prod()**(1.0/len(tunRunTime[i])) )
    tunParamArithmMean.append( np.mean(tunRunTime[i]) )
    #print( i, np.mean(tunRunTime[i]),   np.array(tunRunTime[i]).prod()**(1.0/len(tunRunTime[i]))  )
    
#print( [ (i,tunnParamArithmMean[i]) for i in range(len(tunnParamArithmMean))] )
    
ax.plot( sorted(allParams)[1:], tunParamArithmMean, marker='^', markersize=10 )

ax.xlim = 100

#plt.title(" Speedup for 3 repeated runs and for all the graphs in the tunning set")
plt.ylabel("running time (sec)", fontsize=23)
plt.xlabel("tuning parameter value", fontsize=23)

#the inside, smaller axes
axins = inset_axes( ax,
                    width="50%", # width = 30% of parent_bbox
                    height="50%", # height : 1 inch
                    loc=9,
                  )
axins.alph=0.
axins.plot( sorted(allParams)[5:-1], tunParamArithmMean[4:-1], marker='^', markersize=10)
axins.set_xscale('log', basex=2)
#axins.set_yscale('log', basey=2)

axins.annotate( "best tuning value\nfor $c="+str(tunedParam)+"$", fontsize=15, \
               xy=(tunedParam, tunParamArithmMean[7]+1),\
               xytext=(tunedParam-1000,minTuningTime+15),\
               arrowprops=dict(facecolor='black', shrink=0.05), va = "top", ha="center" )

for axis in ['top','bottom','left','right']:
    axins.spines[axis].set_linewidth(1.5)
    axins.spines[axis].set_edgecolor('0.1')
    
##finish with the inside axes

# set the sub region of the original image
x1, x2, y1, y2 = 1850, 9000, 160, 182
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)

# draw a bbox of the region of the inset axes in the parent axes and
# connecting lines between the bbox and the inset axes area
mark_inset(ax, axins, loc1=3, loc2=4, fc="0.9", ec="0.3", alpha=0.9 )

ax.set_xscale('log', basex=2)
#ax.set_yscale('log', basey=2)

plt.tight_layout()

plt.savefig('../images/tunParam.pdf')
plt.savefig('../images/tunParam.png')
plt.draw()
plt.show()

## Plotting time/size to find the time needed per node 
(if "size"=$n$ , alternatively, "size"=$n+m$).

In [ ]:
#TODO: keep or remove lines?

fig, ax = plt.subplots(figsize=(15,8))
plt.title('Mean running times over number of nodes (time per vertex)')
plt.xlabel("instance size: number of nodes")
plt.ylabel("$\log_{2}$(running time/n)")

print( len(sizesN), timeRKMean )

plt.plot( sizesN, [ t/s for s,t in zip(sizesN,timeKadMean)], marker='o', label='kadabra' )
plt.plot( sizesN[:-missingInst], [ t/s for s,t in zip(sizesN,timeRKMean)][:-missingInst], marker='o', label='rk' )
ax.set_yscale('log', basey=2)
ax.set_xscale('log', basex=2)
plt.legend()
plt.show()

## Normalize with n+m (supposed theoretical running time) for every instance.
We can also normalize with the theoretical running time of the algorithm.
In this case, the result should a near-constant function. 
(Could not find a better theoretical running time in the kadabra paper.)

In [ ]:
#normalize with n*m for every instance
#TODO: keep or remove lines?
fig, ax = plt.subplots(figsize=(15,8))
plt.title('RK time normalised with n+m')
plt.xlabel("instance size: number of nodes") # TODO: here, should it be n*m? (need also to re-sort by n+m)
plt.ylabel("running time/(n+m)")

plt.plot( sizesN, [ t/s for s,t in zip(sizesNplusM,timeKadMean)], marker='o', label='kadabra' )
plt.plot( sizesN[:-1], [ t/s for s,t in zip(sizesNplusM,timeRKMean)][:-1], marker='o', label='rk' )
ax.set_yscale('log', basey=2)
ax.set_xscale('log', basex=2)
plt.legend()
plt.show()

## Normalize size and times
by dividing with the maximum running time for each algorithm and the maximum size.

This is **not** mentioned in the paper and probably **not** of great use since it can give the
wrong impression that kadabra is slower. 
I (harry) think that it may be useful to see how the running time of kadabra does not 
follow the theoreticaly expected behavior; at least not as much as RK.

In [ ]:
plt.figure(figsize=(15,8)) 
plt.title("Normalized running times and instance sizes")

plt.plot( [t/max(timeKadMean) for t in timeKadMean], label='kadabra' )
plt.plot( [t/max(timeRKMean) for t in timeRKMean] , label='rk')
plt.plot( [s/max(sizesNM) for s in sizesNM], label='sizes' )

plt.legend()
plt.show()

## Scatter plot

In [ ]:
print( diams )

from matplotlib.legend_handler import HandlerNpoints

plt.rcParams.update({'font.size': 25})
fig, ax = plt.subplots(figsize=(15,8))
#plt.title("Scatter plot of mean running times")
plt.ylabel("running time (seconds)")
plt.xlabel("instance size: $n+m$")

kadPlt = plt.scatter( sizesNplusM[:-missingInst], timeKadMean[:-missingInst], s=150, label='KADABRA' )
kadPlt2 = plt.scatter( sizesNplusM[-missingInst:], timeKadMean[-missingInst:], s=150, marker='^', label='KADABRA only', color=u'#1f77b4')
RKplt = plt.scatter( sizesNplusM[:], timeRKMean[:], s=150, marker='P', label='RK' )
#kadPlt2, = plt.plot( sizesNplusM[-2:], timeKadMean[-2:] , marker='s', label='kad', color=u'#1f77b4', linestyle='' )

ax.set_yscale('log', basey=2)
ax.set_xscale('log', basex=2)

#uncomment to add labels for the graphs; they are cluttered
#plt.xticks( sorted(sizesNplusM), abbreviate( [x for _, x in sorted(zip(sizesNplusM,evaluationInst), key=lambda pair: pair[0])] ),\
#           rotation=90, fontsize=15  )

plt.legend()
#plt.legend([ (kadPlt, kadPlt2), RKplt], ["Attr A" , "Attr A+B"])
#plt.legend(handler_map={kadPlt2: HandlerLine2D(numpoints=3)})
#plt.legend(handler_map={kadPlt2: HandlerNpoints(marker_pad=0.3, numpoints=3)})

plt.savefig('../images/scatterPlot.pdf')
plt.savefig('../images/scatterPlot.png')
plt.show()

#print(sizesNplusM)
#print(timeKadMean)
#print(timeRKMean)

## Box and violin plots for multiple runs per instance
Extract all running times into a list of lists. This can be plotted directly by boxplot.

The difference of running times for one instance are so small that these plots do not help. 
Especially for smaller values or where we have big differences between instances.
Maybe the number of repeatitions is small (max is 10).
Should we advice something for this case? Use some log scale (this must be done when creating the lists below)?
If we focus on one (or few) instances? For example, instance 19 seems to be a good candidate for both algorithms.

In [ ]:
#TODO: this below could be usefull if used with the positions argument
# where positions = [1,2,4,5,7,8,10,11,...]
# see https://stackoverflow.com/questions/16592222/matplotlib-group-boxplots

#coupling times for every instance
allTimesInterleaved = list()    

for inst in evaluationInst:
    allTimesInterleaved.append( allTimesKad[inst] )
    allTimesInterleaved.append( allTimesRK[inst] )
    
#fig, ax = plt.subplots(figsize=(15, 8))    
#bp = ax.boxplot(
#    allTimesInterleaved[20:]
#    )
#ax.set_yscale('log', basey=2)
#plt.show()

In [ ]:
allTimesKadList = [ allTimesKad[inst] for inst in evaluationInst ]
#print(allInstances[20:])
    
allTimesRKList = [ allTimesRK[inst]  for inst in evaluationInst ]

In [ ]:
plt.rcParams.update({'font.size': 18})
#fig1, ax1 = plt.subplots(figsize=(15, 8))
f, (ax, ax2) = plt.subplots(2, 1, sharex=True, figsize=(15, 8), gridspec_kw = {'height_ratios':[1, 4]})

#ax2.set_title("Kadabra running times")
plt.ylabel("running time (seconds)")

#the box plot and different parameters that control it
# plot some interesting instances, leave the big ones out
ax.boxplot(
    allTimesKadList[10:-2],
    notch=False,
    showmeans=True,
)
ax2.boxplot(
    allTimesKadList[10:-2],
    notch=False,
    showmeans=True,
)

#limit the two subplots
ax.set_ylim(1350, 1500)
ax2.set_ylim(0, 11)

# hide the spines between ax and ax2
ax.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax.xaxis.tick_top()
ax.tick_params(labeltop=False)  # don't put tick labels at the top
ax2.xaxis.tick_bottom()

d = .015  # how big to make the diagonal lines in axes coordinates
# arguments to pass to plot, just so we don't keep repeating them

kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
ax.plot((-d, +d), (-(d+0.07), +d), **kwargs)        # top-left diagonal
ax.plot((1 - d, 1 + d), (-(d+0.07), +d), **kwargs)  # top-right diagonal

kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
ax2.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal

#plt.xticks( [i+1 for i in range(len(allTimesKadList))] , abbreviate(evaluationInst), rotation=40, fontsize=20 )
plt.xticks( [i+1 for i in range(len(allTimesKadList[10:-2]))] , abbreviate(evaluationInst[10:-2]), rotation=40, fontsize=25 )

f.subplots_adjust(hspace=0.07)

plt.savefig('../images/boxPlotKad.pdf')
plt.show()


In [ ]:
plt.rcParams.update({'font.size': 18})
#to cut into 3 parts
#f, (ax, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(15, 8), gridspec_kw = {'height_ratios':[1,2,3]})
f, (ax, ax2) = plt.subplots(2, 1, sharex=True, figsize=(15, 8), gridspec_kw = {'height_ratios':[1,3]})

plt.ylabel("running time (seconds)")

ax.violinplot( allTimesRKList[10:-2] , widths=0.5 )
ax2.violinplot( allTimesRKList[10:-2] , widths=0.5 )
#ax3.violinplot( allTimesRKList[10:] , widths=0.5 )

#limit the two subplots
ax.set_ylim(8480, 8800)
ax2.set_ylim(50, 3050)
#ax3.set_ylim(50, 900)

# hide the spines between ax and ax2
ax.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax.xaxis.tick_top()
ax.tick_params(labeltop=False)  # don't put tick labels at the top
ax2.xaxis.tick_bottom()

d = .015  # how big to make the diagonal lines in axes coordinates
# arguments to pass to plot, just so we don't keep repeating them

kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
ax.plot((-d, +d), (-(d+0.05), +d), **kwargs)        # top-left diagonal
ax.plot((1 - d, 1 + d), (-(d+0.05), +d), **kwargs)  # top-right diagonal

kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
ax2.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal
f.subplots_adjust(hspace=0.07)

plt.xticks( [(i+1) for i in range(len(allTimesKadList[10:-2]))] , abbreviate(evaluationInst[10:-2]), rotation=40, fontsize=25 )

plt.savefig('../images/violinPlotRK.pdf')
plt.show()

## Calculate and plot the variance for repeated runs

In [ ]:
varianceKad = list()
print(len(allTimesKadList))
#for tK, tR in zip(allTimesKadList, allTimesRKList):
    #print( np.var(tK), np.var(tR) )

varianceKad = [ np.var(t)/np.mean(t) for t in allTimesKadList]
varianceRK = [ np.var(t)/np.mean(t) for t in allTimesRKList[:-1]]

fig, ax = plt.subplots(figsize=(15,8))

plt.plot( sizesN, varianceKad , marker='o', label='kadabra') #, linestyle='')
plt.plot( sizesN[:-1], varianceRK , marker='o', label='rk') #, linestyle='' )

#ax.bar( [5*i+1 for i in range(len(sizesN))] , varianceKad, width=0.8, label='kadabra')
#ax.bar( [5*i for i in range(len(sizesN)-2)] , varianceRK, width=0.8, label='rk')

plt.xticks( [ 5*i for i in range(numEvalInst)] , abbreviate(evaluationInst), rotation=40, fontsize=22 )

ax.set_yscale('log', basey=2)
ax.set_xscale('log', basex=2)

plt.legend()
plt.show()

## Violin plots

In [ ]:
fig3, ax3 = plt.subplots(figsize=(15, 8))
ax3.set_title("Kadabra running times for every instance")
#plt.ylabel("running time")

#ax3.boxplot( [ tK-np.mean(tK) for tK in allTimesKadList] )
ax3.boxplot( allTimesKadList , showmeans=True)

plt.xticks( [i+1 for i in range(numEvalInst)] , abbreviate(evaluationInst), rotation=40, fontsize=20 )
#plt.xticks( [i+1 for i in range(len(allTimesKadList[10:]))] , abbreviate(evaluationInst[10:]), rotation=40, fontsize=20 )

ax3.set_yscale('log', basey=2)
#plt.ylim(bottom=0.0000001)

plt.savefig('../images/violinPlotKad.pdf')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
ax.set_title("RK running times for every instance")
#plt.ylabel("running time")

ax.boxplot( allTimesRKList )

plt.xticks( [i+1 for i in range(numEvalInst)] , abbreviate(evaluationInst), rotation=40, fontsize=20 )
#plt.xticks( [i+1 for i in range(len(allTimesKadList[10:]))] , abbreviate(evaluationInst[10:]), rotation=40, fontsize=20 )

ax.set_yscale('log', basey=2)
#plt.ylim(bottom=0.0000001)

#plt.savefig('../images/violinPlotKad.pdf')
plt.show()

In [ ]:
fig3, ax3 = plt.subplots(figsize=(15, 8))
plt.rcParams.update({'font.size': 25})
plt.ylabel("relative deviation in running time")

ax3.boxplot( [ (tRK-np.mean(tRK))/np.mean(tRK) for tRK in allTimesRKList[:-missingInst]] )

yTicks = [i/100 for i in range(-4, 5, 1)]
print(yTicks)

# [:-missingInst] because RK is missing data for a road network
plt.xticks( [i+1 for i in range(numEvalInst+1)][:-missingInst] , abbreviate(evaluationInst[:-missingInst]), rotation=40, fontsize=25 )
plt.yticks( yTicks , [ str(y*100)+'%' for y in yTicks] , fontsize=25 )

plt.savefig('../images/relRunTimeDev_RK.pdf')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
plt.rcParams.update({'font.size': 25})
#ax.set_title("Kadabra")
plt.ylabel("relative deviation in running time")
relDevKad  = [ (tKad-np.mean(tKad))/np.mean(tKad) for tKad in allTimesKadList]

ax.violinplot( relDevKad, showmeans=True )
#ax3.violinplot( allTimesRKList[:-2] )

yTicks = [i/100 for i in range(-16, 17, 4)]

assert( numEvalInst==len(evaluationInst))

plt.xticks( [i+1 for i in range(numEvalInst)] , abbreviate(evaluationInst), rotation=40, fontsize=25 )
plt.yticks( yTicks , [ str(y*100)+'%' for y in yTicks] , fontsize=25 )

#ax3.set_yscale('log', basey=2)
plt.xlim(left=0.5, right=24.5)

plt.savefig('../images/relRunTimeDev_Kad.pdf')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
#ax1.set_title("Kadabra running times for every instance")
plt.ylabel("running time")

numPlots = len(allTimesKadList[10:])

#the box plot and different parameters that control it
bp = ax.boxplot(
    allTimesKadList[10:],
    notch=False,
    showmeans=True,
    boxprops = dict(linewidth=1, markersize=30),
    positions= [2*i for i in range(numPlots)]
)
ax.violinplot( 
    allTimesKadList[10:],
    positions= [2*i+1 for i in range(numPlots)]
)
plt.xlim(right=25.5)
#plt.xticks( [i+1 for i in range(allTimesKadList)] , abbreviate(evaluationInst), rotation=40, fontsize=20 )
plt.xticks( [2*i+0.5 for i in range(numPlots)] , abbreviate(evaluationInst[10:]), rotation=40, fontsize=20 )

#plt.setp(bp['boxes'], color='r')    
plt.savefig('../images/boxViolinKad.pdf')
plt.show()

In [ ]:
#fig, ax = plt.subplots( figsize=(15, 8))
fig, ax = plt.subplots(figsize=(15, 8))

ax = plt.subplot(1,2,1)

plt.rcParams.update({'font.size': 26})

meanpointprops = dict(markersize=15, linestyle='-.', linewidth=3)
medianlineprops = dict(linestyle='-', linewidth=3)

bp = ax.boxplot(
    allTimesKadList[-3],
    showmeans=True,
    #notch=True,
    boxprops = dict(linewidth=3, markersize=30),
    meanprops=meanpointprops,
    medianprops = medianlineprops,
    flierprops=dict(markersize=20),
)

ax = plt.subplot(1,2,2)

parts = ax.violinplot( allTimesRKList[-3], positions=[1.5],
              showmedians=True,
             )
#for pc in parts['bodies']:
    #pc.set_facecolor('#D43F3A')
    #pc.set_edgecolor('black'),
    #pc.set_linewidths(15)
    #pc.set_zorder(4)
    #pc.set_alpha(0.5)
    
plt.xlim( left=0.8, right = 1.8)
plt.xticks( [1.25],  evaluationInst[-3:-2] )
#ax.set_yscale('log', basey=2)

plt.savefig('../images/boxViolion.png')
plt.show()


In [ ]:
import matplotlib.patches as mpatches

fig4, ax4 = plt.subplots(figsize=(15, 8))
#ax4.set_title("RK running times for every instance")
plt.ylabel("running time")

kad_patch = mpatches.Patch(color=u'#1f77b4', label='kadabra')
rk_patch = mpatches.Patch(color=u'#ff7f0e', label='rk')
#print( allTimesRKList[10:] )
ax4.violinplot( allTimesKadList[10:] ) #, positions=sizesNplusM[10:], widths=100)
ax4.violinplot( allTimesRKList[10:-missingInst] ) #, positions=sizesNplusM[10:-2], widths=100)

ax4.set_yscale('log', basey=2)
#ax4.set_xscale('log', basex=2)
#plt.xticks( [i+1 for i in range(numEvalInst)] , abbreviate(evaluationInst), rotation=40, fontsize=20 )
plt.legend(loc=5, handles=[kad_patch, rk_patch])

plt.savefig('../images/violinScatter.pdf')
plt.show()

## Errorbar plot

In [ ]:
plt.figure(figsize=(15,8))
    
lowErrorKad = []
upErrorKad = []

lowErrorRK = []
upErrorRK = []

log_base = 2

for i in range(numEvalInst):
    inst = evaluationInst[i]
    
    instKadRun = [run['run_time'] for run in evalRuns if (run['experiment']=='kadabra' and run['instance']==inst) ]
    #print(inst, instKadRun)
    lowErrorKad.append( timeKadMean[i]-min(instKadRun) )
    upErrorKad.append( max(instKadRun)-timeKadMean[i] )
    
    instRKRun = [run['run_time'] for run in evalRuns if (run['experiment']=='rk' and run['instance']==inst) ]
    if len(instRKRun)==0:
        lowErrorRK.append( timeRKMean[i] )
        upErrorRK.append( timeRKMean[i] )
        continue
    lowErrorRK.append( timeRKMean[i]-min(instRKRun) )
    upErrorRK.append( max(instRKRun)-timeRKMean[i] )

#print(timeKadMean)
fig, ax = plt.subplots(figsize=(15,8))

plt.ylabel("mean running time")
plt.xlabel("number of nodes")

plt.errorbar( [i+1 for i in range(numEvalInst) ][10:], timeKadMean[10:], yerr=[lowErrorKad[10:], upErrorKad[10:]], elinewidth=4, linestyle='--',label='kadabra' )
plt.errorbar( [i+1 for i in range(numEvalInst) ][10:-1], timeRKMean[10:-1], yerr=[lowErrorRK[10:-1], upErrorRK[10:-1]], elinewidth=4, linestyle='--', label='rk')

ax.set_yscale('log', basey=2)
ax.set_xscale('log', basex=2)

#plt.errorbar(sizesN, timeKadMean, yerr=[lowErrorKad, upErrorKad], label='kadabra' )
#plt.errorbar(sizesN, timeRKMean, yerr=[lowErrorRK, upErrorRK], label='rk')

plt.legend()
plt.show()


# Various Visualization Plots - Quality

In [ ]:
#taken from Moritz's Analysis notebook

# Load exact betweenness values for each instance
exactValues = dict()
exactValueDir = "../experiments/output/brandes/"
for instance in cfg.all_instances():
    with open(os.path.join(exactValueDir, instance.filename + '.out-full'), 'r') as f:
        exactValues[instance.filename] = np.loadtxt(f)
        
    #brandes scores are not relative: normalize witn (n-1)(n-2)
        
    if len([ x['n'] for x in evalRuns if x['instance']==instance.shortname] )==0:
        print("No data for", instance.shortname, ", skipping")
        continue
    instSize = [ x['n'] for x in evalRuns if x['instance']==instance.shortname][0]
    
    exactValues[instance.filename] = [ x/((instSize-1)*(instSize-2)) for x in exactValues[instance.filename] ]

In [ ]:
# Load the full betweenness scores for each instance

# - fullResults[inst][algo] is a list with size = the number of repeated runs
# - fullResults[inst][algo][i] is a list of size = size(instance) and contains
#     the betweenness value for every node of inst

fullResults = dict()
for run in cfg.discover_all_runs():
    instance = run.instance.filename
    algo = run.experiment.name
    
    if instance not in [x['instance'] for x in evalRuns]:
        print( "Instance", instance,"not in evaluation set, skipping")
        continue
        
    if not instance in fullResults:
        fullResults[instance] = dict()
    if not algo in fullResults[instance]:
        fullResults[instance][algo] = list()
        
    fullFilename = run.output_file_path('out')+"-full"
    try:
        #print("opening", fullFilename, "for", instance)
        with open(fullFilename, 'r') as f:
            fullResults[instance][algo].append(np.loadtxt(f))
    except:
        pass
    
    #assert sizes agree
    if len(fullResults[instance][algo])==0:
        print(fullFilename)
        print("No data for", instance, "and algo", algo)
        continue
    instSize1 = len(fullResults[instance][algo][0])
    instSize2 = [ x['n'] for x in evalRuns if x['instance']==instance][0]
    
    #TODO: this gives a warning (or shoulb be error?). Ignore for now
    if instSize1!=instSize2:
        print("WARNING: Sizes for instace " + instance + " do not agree. One is "\
              + str(instSize1) + " and the other is " + str(instSize2) )
        #break
    

In [ ]:
# test print
print('number of repeated runs for rk and ca-cit-HepTh', len(fullResults['ca-cit-HepTh']['rk']))
print('number of nodes of ca-cit-HepTh', len(fullResults['ca-cit-HepTh']['rk'][0]))

#number of runs, should be 10 but some have just one
print( 'number of repeated runs for kadabra and ca-AstroPh', len(fullResults['ca-AstroPh']['kadabra']) )
print('number of repeated runs for rk and opsahl-powergrid', len(fullResults['wiki-Vote']['rk']) )

In [ ]:
algos = set([run.experiment.name for run in cfg.discover_all_runs()])
assert( len(algos)==2 )

In [ ]:
avgErrors = dict()
distance = dict()
allRunDiffs = dict()

for algo in algos:
    avgErrors[algo] = []
    distance[algo] = []
    allRunDiffs[algo] = dict()
    
    #WARNING: -1 because we do not have data with rk and one road network
    for instance in evaluationInst[:-missingInst]: #cfg.all_instances():
        allRunDiffs[algo][instance] = []
        
        size = len(exactValues[instance])
        
        #here, the [0] means that we only take the results from the first run
        #diff = [abs(exactValues[instance.filename][i] - fullResults[instance.filename][algo][0][i]) for i in range(size)]
        
        # number of repeated runs for this instance
        numRuns = len(fullResults[instance][algo])
        print("number of runs for instance " + instance + " and algo "+ algo + " is " + str(numRuns) )
        for j in range(numRuns):
            diffN = [abs(exactValues[instance][i] - fullResults[instance][algo][j][i]) for i in range(size)]
            diff = sum(diffN)/size
            #print(diff)
            allRunDiffs[algo][instance].append(diff)
        
        betw1 = exactValues[instance]
        betw2 = fullResults[instance][algo][0]
        dist = np.linalg.norm( betw1-betw2 )
        #print( len(exactValues[instance.filename]))
        
        distance[algo].append(dist)
                
        # average of differences for all repeated runs
        avgErrors[algo].append( sum(allRunDiffs[algo][instance])/numRuns )
        #avgErrors[algo].append(sum(diff)/size)

assert( len(avgErrors['rk'])==nBoth )

In [ ]:
# test prints
print( 'number of algos:' , len(allRunDiffs) )  # number of algos
print( 'number of instances:' , len(allRunDiffs['rk']) )  # number of instances
print( 'number of repeated runs for rk and \'openflights\':',len(allRunDiffs['rk']['openflights']) ) # number of repeated runs for instance and algo
print('4th instance is', evaluationInst[4])
#print( 'all solution differences for rk and ' + instances[11] +': ', allRunDiffs['rk-1t'][instances[11]] ) # all differences for the repeated runs

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

plt.title('Dirrences from exact betweenness values')
meanBtwKad = [sum(allRunDiffs['kadabra'][inst])/len(allRunDiffs['kadabra'][inst]) for inst in evaluationInst[:-missingInst]] 
meanBtwRK = [sum(allRunDiffs['rk'][inst])/len(allRunDiffs['rk'][inst]) for inst in evaluationInst[:-missingInst]] 

#plt.scatter( [i for i in range(numEvalInst)][:-1], meanBtwKad , s=100, label='kadabra' )
#plt.scatter( [i for i in range(numEvalInst)][:-1], meanBtwRK , s=100, marker='P', label='rk' )

ax.bar( [5*i+1 for i in range(numEvalInst)][:-missingInst] , meanBtwKad, width=0.8, label='kadabra')
ax.bar( [5*i for i in range(numEvalInst)][:-missingInst] , meanBtwRK, width=0.8, label='rk')

plt.xticks( [5*i for i in range(numEvalInst)][:-missingInst], abbreviate(evaluationInst), rotation=40, fontsize=20 )
plt.ylim(top=0.0015, bottom=-0.00001)

#ylim for log scale
#plt.ylim(top=0.015, bottom=0.00000002)
#ax.set_yscale('log', basey=2)

plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
plt.rcParams.update({'font.size': 22})

plt.title('Quality: average difference of betweenness compared to ground truth')
plt.xlabel("instance size: number of nodes")
plt.plot( [i for i in range(numEvalInst-missingInst)], avgErrors['kadabra'], marker='o', label='kadabra' )
plt.plot( [i for i in range(numEvalInst-missingInst)], avgErrors['rk'], marker='o', label='rk')
plt.legend()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
plt.rcParams.update({'font.size': 22})

plt.title('Quality: euclidean distance of betweenness vectors from ground truth')
plt.xlabel("instance size: number of nodes")
plt.plot( sizesN[:-missingInst], distance['kadabra'], marker='o', label='kadabra' )
plt.plot( sizesN[:-missingInst], distance['rk'], marker='o', label='rk')
plt.legend()
plt.show()

In [ ]:
#TODO: we do not want to print the betweenness value but the difference from the ground truth

allScoresKadList = list()
allDiffsKad = list()

for inst in evalBothInst:
    allScoresKadList.append( fullResults[inst]['kadabra'] ) 
    allDiffsKad.append( allRunDiffs['rk'][inst] )
    
print( len(allDiffsKad) ) 
print( allDiffsKad[5])
#allTimesRKList = list()
#for i in range(numInstances):
    #if timeRKMean[i] > 2500:
#    allTimesRKList.append( allTimesRK[instances[i]] ) 


In [ ]:
fig2, ax2 = plt.subplots(figsize=(15, 8))
ax2.set_title("betweenness scores")

ax2.boxplot(
    #allScoresKadList[0],
    allDiffsKad[0],
    notch=True,
    sym='o',
)

plt.show()

In [ ]:
fig3, ax3 = plt.subplots(figsize=(15, 8))
ax3.set_title("betweenness scores")

#plt.scatter( [i for i in range(numInstances)], [exactValues[inst] for inst in instances], s=50, label='exact' )

ax3.boxplot(
    [exactValues[inst] for inst in evaluationInst][0],
    #allDiffsKad[0],
    notch=False,
    #sym='o',
)

plt.show()


## Function to write macros for numerical values that are used in main.tex

In [ ]:
def make_macro(file, name, value):
    file.write( '\\newcommand{\\' + name + '}{' + value + '}\n' )

In [ ]:
# the file to store the macros
macrosF = open('../numerical_macros.tex', 'w')

make_macro( macrosF, "tunParamValue", str(tunedParam) )
make_macro( macrosF, "minTunParamTime", str(minTuningTime) )

make_macro( macrosF, "speedUpGeomMean", str(geomMean) )
make_macro( macrosF, "speedUpMin", str(minSpeedUp) )
make_macro( macrosF, "minSpeedUpInst", minSpeedUpInst )
make_macro( macrosF, "speedUpMax", str(maxSpeedUp) )
make_macro( macrosF, "maxSpeedUpInst", maxSpeedUpInst )

strOnlyKad = ""
for s in onlyKadInst:
    print(s)
    strOnlyKad += s + " ("+abbreviate([s])[0]+ "), "
strOnlyKad = strOnlyKad[:-2] #remove the comma and space

make_macro( macrosF, "onlyKadInstances", strOnlyKad )

macrosF.close()